In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
# Helper functions for metrics
def calculate_metrics(df):
    """Calculates annualized volatility and Sharpe ratio for given returns."""
    daily_returns = df.pct_change()
    annualized_volatility = daily_returns.std() * np.sqrt(252)
    sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)
    return annualized_volatility, sharpe_ratio

def download_data(tickers, start_date, end_date):
    """Downloads stock data and checks for missing data."""
    data = yf.download(tickers, start=start_date, end=end_date)
    if data.empty:
        raise ValueError("No data returned. Check your tickers and date range.")
    return data['Adj Close']

def portfolio_allocation_plot(tickers_weights):
    """Generates a pie chart for portfolio allocation."""
    fig = px.pie(values=tickers_weights.values(), names=tickers_weights.keys(), title='Portfolio Allocation')
    return fig

def calculate_weighted_returns(df, weights):
    """Calculates weighted returns for the portfolio."""
    return df.pct_change().dot(weights)

def portfolio_vs_benchmark(portfolio_returns, benchmark_returns):
    """Generates comparison plots for portfolio and benchmark."""
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Portfolio Cumulative Returns", "Risk-Reward Analysis"))
    
    # Cumulative returns
    portfolio_cum_returns = (1 + portfolio_returns).cumprod() - 1
    benchmark_cum_returns = (1 + benchmark_returns).cumprod() - 1
    fig.add_trace(go.Scatter(x=portfolio_cum_returns.index, y=portfolio_cum_returns, name='Portfolio'), row=1, col=1)
    fig.add_trace(go.Scatter(x=benchmark_cum_returns.index, y=benchmark_cum_returns, name='Benchmark'), row=1, col=1)
    
    # Risk-reward
    port_vol, port_sharpe = calculate_metrics(portfolio_returns)
    bench_vol, bench_sharpe = calculate_metrics(benchmark_returns)
    fig.add_trace(go.Scatter(x=[port_vol], y=[port_sharpe], mode='markers', name='Portfolio', marker=dict(color='blue', size=10)), row=1, col=2)
    fig.add_trace(go.Scatter(x=[bench_vol], y=[bench_sharpe], mode='markers', name='Benchmark', marker=dict(color='red', size=10)), row=1, col=2)
    
    fig.update_layout(title_text="Portfolio vs Benchmark")
    return fig

def main_portfolio_function(tickers_and_values, start_date, end_date, benchmark):
    """Main function to generate all plots and analyses."""
    tickers = list(tickers_and_values.keys())
    data = download_data(tickers + [benchmark], start_date, end_date)
    
    # Check for missing data not included, assume data is complete for simplicity
    
    # Portfolio allocation
    total_value = sum(tickers_and_values.values())
    weights = np.array([value / total_value for value in tickers_and_values.values()])
    
    portfolio_data = data[tickers]
    benchmark_data = data[benchmark]
    
    # Calculate returns
    portfolio_returns = calculate_weighted_returns(portfolio_data, weights)
    benchmark_returns = benchmark_data.pct_change()
    
    # Generate plots
    allocation_fig = portfolio_allocation_plot(tickers_and_values)
    comparison_fig = portfolio_vs_benchmark(portfolio_returns, benchmark_returns)
    
    # Display or return plots
    allocation_fig.show()
    comparison_fig.show()

In [3]:
# Define your portfolio and run
tickers_and_values = {
    'AAPL': 2600,
    'NVDA': 700,
    'MSFT': 1800
}
main_portfolio_function(tickers_and_values, '2015-09-02', '2022-11-08', '^GSPC')


[*********************100%%**********************]  4 of 4 completed
C:\Users\Fábio Coatis\AppData\Roaming\Python\Python312\site-packages\pandas\core\nanops.py:1010: RuntimeWarning:

invalid value encountered in subtract

